# Surface Event Pick Time

This is a modified version of the surface-event location+directivity analysis that Francesca Skene ( fskene@uw.edu), originally created by her in 7/22/22, who started as an undergraduate student at UW. This is marine denolle's version. It includes:
* Waveform download for each event on each volcano given the PNSN pick times of "su" events.
* Data pre-processing to trim the data within 2-12 Hz and remove outliers.
* phase picking using transfer-learned model (Ni et al, 2023)
* event location using 1D grid search
* directivity measurements (velocity and direction) using Doppler effects.
* gathering of the data into a CSV data frame.

Updated 01/24/2024
Marine Denolle
(mdenolle@uw.edu)

Import Modules

In [1]:
# import sys
# sys.path.append('/data/wsd01/pnwstore/')
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import obspy
from obspy.core import UTCDateTime
from obspy.clients.fdsn.client import Client

import scipy
from scipy import optimize
from scipy.optimize import curve_fit
from geopy import distance
from datetime import datetime
from utils import *
from mbf_elep_func import *
import torch
plt.rcParams.update({'font.size': 10})


import seisbench.models as sbm
device = torch.device("cpu")

# from ELEP.elep.ensemble_statistics import ensemble_statistics
from ELEP.elep.ensemble_coherence import ensemble_semblance 
# from ELEP.elep.ensemble_learners import ensemble_regressor_cnn
from ELEP.elep import mbf, mbf_utils
from ELEP.elep import trigger_func

from ELEP.elep.mbf_utils import make_LogFq, make_LinFq, rec_filter_coeff, create_obspy_trace
from ELEP.elep.mbf import MB_filter as MBF

Parameters

In [2]:
# define clients to download the station data
# client = WaveformClient() # we ignore PNWdatastore for now
client2 = Client('IRIS')

t_before = 15 #number of seconds before pick time
# t_after = 15 #number of seconds after pick time
t_before_raw = 1200 #number of seconds before pick time before removing instrumental response
# t_after_raw = 1200 #number of seconds after pick time before removing instrumental response
fs = 40 #sampling rate that all waveforms are resampled to
window = 150 #window length of the signal (this will help with phase picking with EqT next). 
# Use 150 seconds @ 40 Hz gives 6001 points. 
pr = 98 #percentile
thr = 7 #SNR threshold
station_distance_threshold = 25
pi = np.pi
v_s = 1000 #shear wave velocity at the surface

# range of dates that we are looking at
t_beginning = UTCDateTime(2001,1,1,0,0,0) 
t_end = UTCDateTime(2023,8,2,23,59)

smooth_length = 20 # constant for smoothing the waveform envelopes
low_cut = 1 #low frequency threshold
high_cut = 15 #high frequency threshold
# az_thr = 1000 #threshold of distance in meters from source location
# step = 100 #step every 100 m
# t_step = 1 #step every second
# ratio = 5.6915196 #used to define the grid 
# colors = list(plt.cm.tab10(np.arange(10)))*3
# radius = 6371e3 # radius of the earth

## Volcano - Station Information

In [3]:
#this data includes all stations within 50km of each volcano and the lat, lon, elev of each station
df = pd.read_csv('../data/station/Volcano_Metadata_50km.csv')

## PNSN SU Pick information

In [4]:
f1 = pd.read_csv("../data/events/su_picks.txt",sep="|")
f1.head()
print(f1.keys())

Index(['date', 'net', 'sta', 'location', 'seedchan', 'iphase', 'quality',
       'orid', 'etype', 'evid'],
      dtype='object')


In [5]:
# clean up the spaces in the file
format='%Y/%m/%d %H:%M:%S'
test=f1["date"].values.tolist()
start_time_temp = [  datetime.strptime(x.strip(),'%Y/%m/%d %H:%M:%S') for x in f1["date"].values.tolist()]
# # Ignore events prior to t_beginning
ik=np.where(np.array(start_time_temp)>datetime(2001,1,1))[0][0]

# select only net, sta, evid, startime for event past the start date.

start_time = start_time_temp[ik:]
net=[ x.strip() for x in f1["net"].values.tolist()][ik:]
sta=[ x.strip() for x in f1["sta"].values.tolist()][ik:]
evt_id=[ x for x in f1["orid"].values.tolist()][ik:]
all_stas=set(sta)

## ML Models

In [6]:
# import os
# os.makedirs("/Users/marinedenolle/.seisbench/models/v3/eqtransformer",exist_ok=True)

In [7]:
# !wget https://github.com/congcy/ELEP/raw/main/docs/tutorials/data/pnw.pt.v1 -O ~/.seisbench/models/v3/eqtransformer/pnw.pt.v1
# !wget https://github.com/congcy/ELEP/raw/main/docs/tutorials/data/pnw.json.v1 -O ~/.seisbench/models/v3/eqtransformer/pnw.json.v1

In [8]:
# download models
list_models_name = ["pnw","ethz","instance","scedc","stead","geofon"]
pn_pnw_model = sbm.EQTransformer.from_pretrained('pnw')
pn_ethz_model = sbm.EQTransformer.from_pretrained("ethz")
pn_instance_model = sbm.EQTransformer.from_pretrained("instance")
pn_scedc_model = sbm.EQTransformer.from_pretrained("scedc")
pn_stead_model = sbm.EQTransformer.from_pretrained("stead")
pn_geofon_model = sbm.EQTransformer.from_pretrained("geofon")
# pn_neic_model = sbm.EQTransformer.from_pretrained("neic")

list_models = [pn_pnw_model, pn_ethz_model, pn_instance_model, pn_scedc_model, pn_stead_model, pn_geofon_model]

pn_pnw_model.to(device);
pn_ethz_model.to(device);
pn_scedc_model.to(device);
# pn_neic_model.to(device);
pn_geofon_model.to(device);
pn_stead_model.to(device);
pn_instance_model.to(device);

In [9]:
paras_semblance = {'dt':0.025, 'semblance_order':4, 'window_flag':True, 
                   'semblance_win':0.5, 'weight_flag':'max'}
p_thrd, s_thrd = 0.01, 0.05

fqmin = low_cut
fqmax = high_cut
dt = 0.025; fs = 40
nfqs = 10
nt = 6000; nc = 3
fq_list = make_LogFq(fqmin, fqmax, dt, nfqs)
coeff_HP, coeff_LP = rec_filter_coeff(fq_list, dt)
MBF_paras = {'f_min':fqmin, 'f_max':fqmax, 'nfqs':nfqs, 'frequencies':fq_list, 'CN_HP':coeff_HP, 'CN_LP':coeff_LP, \
    'dt':dt, 'fs':fs, 'nt':nt, 'nc':nc, 'npoles': 2}

# Pick waveforms

In step, we repick all the waveforms, including those already picked by the network analysis.

In [11]:
from matplotlib.backends.backend_pdf import PdfPages
# pdf = PdfPages('../plots/MLPicks_test_November_2023.pdf')
associated_volcano = "Mt_Rainier"
dff=[]
for n in range(len(evt_id)): 
    if start_time[n]<datetime(2023,11,1):continue   
    event_ID = str(evt_id[n])
    otime = UTCDateTime(start_time[n])
    networks=net[n]
    stations=sta[n]
    try:
        associated_volcano = df[df['Station']== sta[n]]['Volcano_Name'].values[0]
    except: 
        pass
    if associated_volcano!="Mt_Rainier":continue


    #get info for stations within 50km of volcano that event ocurred at
    stations = df[df['Volcano_Name'] == associated_volcano]['Station'].values.tolist()
    networks = df[df['Volcano_Name'] == associated_volcano]['Network'].values.tolist()
    latitudes = df[df['Volcano_Name'] == associated_volcano]['Latitude'].values.tolist()
    longitudes = df[df['Volcano_Name'] == associated_volcano]['Longitude'].values.tolist()
    elevations = df[df['Volcano_Name']== associated_volcano]['Elevation'].values.tolist()

    print("there are ",len(stations),"stations at",associated_volcano)           
    print(event_ID,otime)

#################### WAVEFORM DOWNLOAD #######################
    #Download all waveforms for that event based on stations and times
    bulk = [] 
    for m in range(0, len(networks)):
        bulk.append([networks[m], stations[m], '*', '*Z', otime-t_before_raw, otime+t_before_raw])
    # try:
    st = client2.get_waveforms_bulk(bulk)
    if (len(st))<3:continue # if there are fewer than 3 stations, skip the event
    st = resample(st,fs)  #resampling the data to 40Hz for each trace

    evt_data = obspy.Stream()
    snr=[]
    stas=[]
    nets=[]
    centroid_time = []
    data_env_dict = {}
    duration = []

    # #Keeping all traces for one event with channel z, SNR>10, and bandpassed between 2-12Hz
    # ,nets,max_amp_times,durations,data_env_dict,t_diff = [],[],[],[],[],[],[],{},{}
    for i,ii in enumerate(st):
        ii.detrend(type = 'demean')
        ii.filter('bandpass',freqmin=low_cut,freqmax=high_cut,corners=2,zerophase=True)
        # trim the data and noise window to exactly 6000 points
        signal_window = ii.copy()
        noise_window = ii.copy()
        signal_window.trim(otime - t_before, otime - t_before + window) # trim the signal at the first pick time of the PNSN data, with loose 40s before
        noise_window.trim(otime - window -t_before, otime - t_before) # noise window of the same length
        if  len(signal_window.data)<=10 or  len(noise_window.data)<=10: continue # skip if no data
        
        # if not np.percentile(np.abs(signal_window.data),pr):continue # skip if max amplitude is zero
        snr1 = (20 * np.log(np.percentile(np.abs(signal_window.data),pr) 
                        / np.percentile(np.abs(noise_window.data),pr))/np.log(10))

        if snr1<thr: # and 100<max_amp_time<200:
            st.remove(ii)
            continue

################# ENVELOPE, CENTROID, DURATION #######################
        # enveloping the data 
        data_envelope = obspy.signal.filter.envelope(signal_window.data)
        data_envelope = obspy.signal.util.smooth(data_envelope, smooth_length)

        data_env_dict[ii.stats.network+'.'+ii.stats.station]= data_envelope/max(np.abs(data_envelope))

        # finding the time of max amplitude of each event
        # signal_window is windowed at otime-t_v before the PNSN pick time
        crap = np.argmax(np.abs(data_envelope))+t_before # time of max amplitude relative to otime
        centroid_time.append(crap/fs)

        # find duration as data starting with the "origin time" and ending when the envelope falls below the mean noise
        noise_envelope = obspy.signal.filter.envelope(noise_window.data)
        data_envelope = obspy.signal.util.smooth(data_envelope, smooth_length)
        mean_noise = np.mean(noise_envelope)
        
        mmax = np.max(np.cumsum(data_envelope**4))
        crap = np.where( np.cumsum(data_envelope**4) <= 0.999*mmax)[0][-1]
        duration.append(crap/fs-t_before)
        # duration.append(np.sum(data_envelope>mean_noise)/fs)

        stas.append(ii.stats.station)
        nets.append(ii.stats.network)
        snr.append(snr1)
        evt_data.append(signal_window)

        # t = evt_data.select(station=stas[i])[0].times()
        # plt.plot(t-t_before,np.cumsum(data_envelope**4))
        # break
    if len(stas)<3:continue



################### ELEP #######################
    
        # test the new function
    smb_peak= apply_elep(evt_data, stas, \
            list_models, MBF_paras, paras_semblance, t_before)
    print(stas)
    print(smb_peak-t_before) 
    print(np.asarray(centroid_time))  



    ############# DURATION ESTIMATE #################
    

    ############## PEAK FREQUENCY MEASUREMENTS ############
    # Given the approximate measurement of duration, window the signal windows around that
    # then measure peak frequency so that there is less noise in it.
    # perform this on the Z component only.
    
    char_freq, sharp_weight= [],[]
    fig,ax = plt.subplots(1,1,figsize=(11,8), dpi = 200)
    for ii,i in enumerate(evt_data):
        data = np.zeros(200*fs)
        crap=i.copy()
        otime1 = crap.stats.starttime + smb_peak[ii] # pick time
        crap.trim(otime1  - 10, otime1 + 2*duration[ii] + 10) # window the data around the pick time
        crap.taper(max_percentage=0.01,max_length=20)

        data[:len(crap.data)] = crap.data #*100
        f,psd=scipy.signal.welch(data,fs=fs,nperseg=81,noverlap=4)
        #just get the frequencies within the filter band
        above_low_cut = [f>low_cut]
        below_high_cut = [f<high_cut]
        in_band = np.logical_and(above_low_cut,below_high_cut)[0]
        f = f[in_band]
        psd = psd[in_band]

        # calculate characteristic frequency and report
        char_freq_max = f[np.argmax(psd)]
        char_freq_mean= np.sum(psd*f)/np.sum(psd)
        psd_cumsum = np.cumsum(psd)
        psd_sum = np.sum(psd)
        char_freq_median = f[np.argmin(np.abs(psd_cumsum-psd_sum/2))]
        char_freq.append(char_freq_mean)

        print(stas[ii],char_freq_mean)
        plt.rcParams.update({'font.size': 20})
        p=ax.plot(f,psd,label=stas[ii],linewidth=2)
        cc = p[0].get_color()
        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.grid('True')
        ax.set_xlabel('Frequency [Hz]')
        ax.set_ylabel('PSD [$(mm/s)^2$/Hz]')
        ax.vlines(char_freq_mean,ymin=np.min(psd)/10,ymax=np.max(psd)*10,linestyle="--",colors=cc)

    #             # weighting the data by the spikiness of the PSD vs frequency graphs
        ratio = (np.mean(psd)/np.max(psd))
        sharp_weight.append(int(1/(ratio**2)*20))


        ############# KEEP DATA #######################


#         if not max(smb_peak.shape):continue
    ddict = {'otime':otime, 'nets':nets, 'stas':stas,  'snr':snr, 'smb_peak': smb_peak, 'centroid_time': centroid_time ,
             'duration':duration, 'char_freq':char_freq, 'sharp_weight':sharp_weight, 'volcano':associated_volcano, 'event_ID':event_ID}
    print(ddict)
    if not np.any(dff):
        dff = pd.DataFrame.from_dict(ddict)
    else:
        dff=pd.concat([dff,pd.DataFrame.from_dict(ddict)],ignore_index=True)
    print(dff)


# ################# PLOT PICKS #######################
#         # ###############

# ################ SAVE PICKS ##############################
#         print(otime)
#         print("net,stas,snr,picks from ELEP,centroid pick time")
#         break
#         for i in len(stas):
#                 print(nets[i],stas[i],snr[i],smb_peak[i]-t_before,centroid_time[i])

#         # print(smb_peak-t_before) # this is the difference between the pick time and the PNSN pick time
#         break

#         # pick_mse[n] = np.sqrt(np.sum(smb_peak[:]-t_before)**2/len(stas))
#         # pick_mae[n] = np.sum(np.abs(smb_peak[:]-t_before))/len(stas)

#             ## plot figure

    # fig = plt.figure(figsize = (11,8), dpi=200)
    # fig.suptitle(str(otime)+" "+associated_volcano)
    # ax = plt.subplot(1,1,1)
    # iplot = 0
    # for i in range(len(stas)):
    #     data = evt_data.select(station=stas[i])[0].data
    #     max1 = np.max(np.abs(data))
    #     t = evt_data.select(station=stas[i])[0].times()
    #     ax.plot(t-t_before,data/max1+iplot*1.5,linewidth=0.5)
    #     if np.any(data_env_dict[nets[i]+'.'+stas[i]]):
    #         ax.plot(t-t_before,data_env_dict[nets[i]+'.'+stas[i]]+iplot*1.5,'k',linewidth=1)
    #     # ax.plot(smb_peak[i]-t_before,iplot*1.5,'r*',markersize=5)  
    #     ax.plot(centroid_time[i]-t_before,iplot*1.5,'k*',markersize=5)  
    #     ax.set_yticks([])
    #     plt.text(-15, iplot*1.5+0.5, stas[i])
    #     print(stas[i],snr[i])
    #     # if i==ista:
    #     # err_title=("%s %2.2f (s) error in picks"%(stas[i],smb_peak[i]-t_before))
    #     # plt.text(60, iplot*1.5+0.5,err_title,color='r')
    #     plt.vlines(smb_peak[i]-t_before,iplot*1.5-1.,iplot*1.5+1.,'r')
    #     print(duration[i],char_freq[i],sharp_weight[i])
    #     # print(stas[i],smb_peak[i]-t_before)
    #     iplot+=1
    # # plt.grid(True)
    # ax.set_xlim([-t_before,80])
    # ax.set_xlabel('time (seconds) relative to PNSN picks')
    # plt.show()
    # pdf.savefig(fig)
    # plt.clf()
    # del fig



#         # pick_mse[n] = np.sqrt(np.sum(smb_peak[:]-t_before)**2/len(stas))
#         # pick_mae[n] = np.sum(np.abs(smb_peak[:]-t_before))/len(stas)
#         # print(pick_mse[n],pick_mae[n] )
    
    # break
        # if n > 10000: break
#     except:
#         pass
# pdf.close()
dff.to_csv("../data/events/MLPicks_test_November_2023.csv")

there are  58 stations at Mt_Rainier
3247953 2023-11-05T07:05:01.000000Z
['ARAT', 'CARB', 'CRBN', 'OBSR', 'PANH', 'PARA', 'PR04', 'PR05', 'RUSH', 'SIFT', 'VOIT', 'FMW', 'LO2', 'LON', 'PUPY', 'RCM', 'RCS', 'RER']
[-15.      7.05    6.      2.      1.225   1.675   4.5     2.475   2.775
   3.425   4.125   1.225   4.125   2.7   -15.     -0.65    0.125   1.275]
[ 24.2    35.05   35.125  32.95   24.5    24.7    30.675  32.625  31.2
  28.1    29.35   24.275  27.475  27.375 112.125  22.025  27.275  27.6  ]
ARAT 2.5141985294438207
CARB 2.596757726092977
CRBN 3.0243057356661756
OBSR 3.018218714502247
PANH 1.964623430601901
PARA 2.3744014700789617
PR04 2.8093477187567366
PR05 2.9134786294683046
RUSH 2.200016402701774
SIFT 2.2048149308481015
VOIT 2.298395358877981
FMW 2.6885993789347022
LO2 2.8423569365367354
LON 3.020713047514075
PUPY 9.331805900879564
RCM 3.1369340692980385
RCS 3.3924350762303495
RER 2.829345032478944
{'otime': UTCDateTime(2023, 11, 5, 7, 5, 1), 'nets': ['CC', 'CC', 'CC', 'CC', 

/var/folders/j4/3rbzxjb16kv66g_p4wkzq16h0000gn/T/ipykernel_25401/471678460.py:122: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig,ax = plt.subplots(1,1,figsize=(11,8), dpi = 200)


['ARAT', 'CARB', 'OBSR', 'PANH', 'PARA', 'PR04', 'PR05', 'RUSH', 'SIFT', 'VOIT', 'FMW', 'LO2', 'LON', 'RCM', 'RCS', 'RER']
[  2.225  11.225  -0.3    -0.1     0.15    9.65  -15.      6.425   7.4
   2.725   0.45    3.825   3.975  -1.575  -1.55   -0.25 ]
[67.025 75.5   67.875 66.15  64.85  66.25  69.55  73.725 67.7   70.025
 68.2   67.875 67.825 64.3   67.05  66.475]
ARAT 3.4987462904636564
CARB 3.55313495703135
OBSR 3.72581604761941
PANH 3.805391292903223
PARA 2.9503265734788884
PR04 4.213102529977315
PR05 3.9890650120181848
RUSH 3.551021006143108
SIFT 2.842335479628805
VOIT 2.2053488183785994
FMW 3.9111333107271777
LO2 5.491323164392307
LON 5.102485047380435
RCM 3.7825284372023367
RCS 5.150378073004321
RER 4.052268372690668
{'otime': UTCDateTime(2023, 11, 23, 12, 46, 20), 'nets': ['CC', 'CC', 'CC', 'CC', 'CC', 'CC', 'CC', 'CC', 'CC', 'CC', 'UW', 'UW', 'UW', 'UW', 'UW', 'UW'], 'stas': ['ARAT', 'CARB', 'OBSR', 'PANH', 'PARA', 'PR04', 'PR05', 'RUSH', 'SIFT', 'VOIT', 'FMW', 'LO2', 'LON', 'R

In [ ]:
p[0].get_color()

'#1f77b4'

In [ ]:
np.percentile(np.abs(noise_window.data),pr)/np.log(10)

In [ ]:
np.percentile(np.abs(noise_window.data),pr)

In [ ]:
np.any(noise_window.data)

In [ ]:
ddict = {'otime':otime, 'nets':nets, 'stas':stas,  'snr':snr, 'smb_peak': smb_peak, 'centroid_time': centroid_time }
print(ddict)
ff = pd.DataFrame.from_dict(ddict)
ff.head()

pd.concat([ff,pd.DataFrame.from_dict(ddict)],ignore_index=True)

In [ ]:
df[df["Volcano_Name"]=="Mt_Rainier"]

In [ ]:

# for n in range(11649,11660):    
#     event_ID = str(evt_id[n])
#     otime = UTCDateTime(start_time[n])
#     print(event_ID,otime)
#     if net != 'CN' and evt_id[n]!=evt_id[n-1]:
#         reference = str(net[n]+'.'+sta[n])
#         try:
#             associated_volcano = df[df['Station']== sta[n]]['Volcano_Name'].values[0]
#         except: 
#             pass
#         print(associated_volcano)
            
#         #get info for stations within 50km of volcano that event ocurred at
#         stations = df[df['Volcano_Name'] == associated_volcano]['Station'].values.tolist()
#         networks = df[df['Volcano_Name'] == associated_volcano]['Network'].values.tolist()
#         latitudes = df[df['Volcano_Name'] == associated_volcano]['Latitude'].values.tolist()
#         longitudes = df[df['Volcano_Name'] == associated_volcano]['Longitude'].values.tolist()
#         elevations = df[df['Volcano_Name']== associated_volcano]['Elevation'].values.tolist()

#         if stations.count("LON")>0 and stations.count("LO2")>0:
#             index = stations.index("LO2")
#             del stations[index]
#             del networks[index]
#             del latitudes[index]
#             del longitudes[index]
#             del elevations[index]


# #################### WAVEFORM DOWNLOAD #######################
#         #Download all waveforms for that event based on stations and times
#         bulk = [] 
#         for m in range(0, len(networks)):
#             bulk.append([networks[m], stations[m], '*', '*H*', otime-t_before_raw, otime+t_after_raw])
#         st = client2.get_waveforms_bulk(bulk)

#         #remove unwanted data
#         for tr in st:
#             cha = tr.stats.channel
#             try:
#                 if len(tr.data)/tr.stats.sampling_rate < 239.9:
#                     st.remove(tr)
#             except:
#                 pass

#         #resampling the data to 40Hz for each trace
#         st = resample(st,fs) 
#         # #Plotting all traces for one event with channel z, SNR>10, and bandpasses between 2-12Hz
#         SNR,SNR_weight, no_weight,stas,nets,max_amp_times,durations,data_env_dict,t_diff = [],[],[],[],[],[],[],{},{}
#         fig = plt.figure()#figsize = (11,8), dpi=200)
#         fig.suptitle('evtID:UW'+ event_ID+associated_volcano)
#         evt_data = obspy.Stream()
#         # plt.rcParams.update({'font.size': 20})
#         ax = plt.subplot(1,1,1)
#         iplot = 0
#         # zz = evt_data.select(component="Z")
#         for i,ii in enumerate(st):
#             network = ii.stats.network
#             station = ii.stats.station
#             ii.detrend(type = 'demean')
#             ii.filter('bandpass',freqmin=low_cut,freqmax=high_cut,corners=2,zerophase=True)
#             cha = ii.stats.channel
#             # ii.trim(otime,otime+window)
#             starttime = ii.stats.starttime
#             signal_window = ii.copy()
#             noise_window = ii.copy()
#             signal_window.trim(otime - 10, otime - 10 + window) # trim the signal at the first pick time of the PNSN data, with loose 40s before
#             noise_window.trim(otime - window -10, otime - 10) # noise window of the same length
#             snr = (20 * np.log(np.percentile(np.abs(signal_window.data),pr) 
#                             / np.percentile(np.abs(noise_window.data),pr))/np.log(10))
#             max_amp_time = np.argmax(noise_window.data)/fs
#             # print(snr,max_amp_time)
#             # signal_window.plot()x
#             if snr<thr: # and 100<max_amp_time<200:
#                 st.remove(ii)
#                 continue
#             t = signal_window.times()
#             t_diff[network+'.'+station] = starttime-otime 
#             # enveloping the data 
#             data_envelope = obspy.signal.filter.envelope(signal_window.data)
#             data_envelope /= np.max(data_envelope)
#             # data_envelope += iplot*1.5
#             # finding the time of max amplitude of each event
#             max_amp_times.append(max_amp_time)
#             max_amp = np.max(ii.data)      
#             # creating envelope data dictionary to calculate picktimes
#             data_envelope = obspy.signal.util.smooth(data_envelope, smooth_length)
#             # kurt = rt.signal.kurtosis(signal_window.taper(max_percentage=0.05))
#             # kurt /= np.max(kurt)
#             data_env_dict[network+'.'+station]= data_envelope
#             # b,e = 115,150
#             if cha[-1]=="Z":
#                 ax.plot(t,signal_window.data/np.max(np.abs(signal_window.data))+iplot*1.5)
#                 ax.plot(t,data_envelope+iplot*1.5, color = 'k')
#                 # ax1.plot(t,kurt+iplot*1.5, color = 'r')
#                 # ax1.plot(t[(t_before_raw+b)*fs:(t_before_raw+e)*fs], data_envelope, color = 'k')
#                 # ax1.plot(t[b*fs:e*fs],ii.data[b*fs:e*fs]/np.max(np.abs(ii.data))+iplot*1.5)
#                 # ax1.plot(t[115*fs:150*fs], data_envelope, color = 'k')
#                 ax.set_xlabel('time (seconds)')
#                 # ax1.set_xlim([b+t_before_raw,e+t_before_raw])
#                 ax.set_yticks([])
#                 # plt.text(t[e*fs], iplot*1.5, 'SNR:'+str(int(snr)))
#                 # plt.text(t[b*fs], iplot*1.5, station)

#                 iplot = iplot+1
#             stas.append(ii.stats.station)
#             nets.append(ii.stats.network)
#             SNR.append(snr)
#             SNR_weight.append(int(snr))
#             no_weight.append(1)
#             evt_data.append(signal_window)
#     # impose at least 4 stations
#     if len(evt_data)<4:  
#         continue

#     sta_available = np.unique(np.array(stas))
#     print(sta_available)
    

# ## Get an approximative measurement of duration taking the highest SNR data
#     ik=np.argmax(SNR)
#     crap =np.cumsum(np.abs(evt_data[ik].data))
#     maxcrap = crap[-1]
#     Td=len(np.where((crap>=0.02*crap[-1])&(crap<=0.95*crap[-1]))[0])/fs
#     print("The approximate duration of this event is %f s "%Td)



# ############## PEAK FREQUENCY MEASUREMENTS ############
# # Given the approximate measurement of duration, window the signal windows around that
# # then measure peak frequency so that there is less noise in it.
# # perform this on the Z component only.
#     data2measure_peak_frequency=evt_data.copy()
#     print(data2measure_peak_frequency[3:6])
#     data2measure_peak_frequency=data2measure_peak_frequency.select(component="Z")
#     print(data2measure_peak_frequency[3:6])
#     data2measure_peak_frequency.taper(max_percentage=0.01,max_length=20)
#     # evt_data.trim(starttime=otime-20,endtime=otime+Td+20) 
#     #         # make plot of spectra
#     char_freq, sharp_weight= [],[]
#     fig1,ax1 = plt.subplots(1,1,figsize=(11,7), dpi = 200)
#     for i in range(len(data2measure_peak_frequency)):
#         data = data2measure_peak_frequency[i].data #*100
#         f,psd=scipy.signal.welch(data,fs=fs,nperseg=81,noverlap=4)
#         #just get the frequencies within the filter band
#         above_low_cut = [f>low_cut]
#         below_high_cut = [f<high_cut]
#         in_band = np.logical_and(above_low_cut,below_high_cut)[0]
#         f = f[in_band]
#         psd = psd[in_band]

#         # calculate characteristic frequency and report
#         char_freq_max = f[np.argmax(psd)]
#         char_freq_mean= np.sum(psd*f)/np.sum(psd)
#         psd_cumsum = np.cumsum(psd)
#         psd_sum = np.sum(psd)
#         char_freq_median = f[np.argmin(np.abs(psd_cumsum-psd_sum/2))]
#         char_freq.append(char_freq_mean)

# #             plt.rcParams.update({'font.size': 20})
#         ax1.plot(f,psd,label=stas[i],linewidth=2)
#         ax1.set_xscale('log')
#         ax1.set_yscale('log')
#         ax1.grid('True')
#         ax1.set_xlabel('Frequency [Hz]')
#         ax1.set_ylabel('PSD [$(mm/s)^2$/Hz]')
#         ax1.vlines(char_freq_mean,ymin=np.min(psd)/10,ymax=np.max(psd)*10,linestyle="--",colors=colors[i])

# #             # weighting the data by the spikiness of the PSD vs frequency graphs
#         ratio = (np.mean(psd)/np.max(psd))
#         sharp_weight.append(int(1/(ratio**2)*20))

# #         ax.legend() 
# #         plt.savefig('psd'+event_ID+associated_volcano+'.png')

#         # lats, lons, elevs, r, theta = ([] for i in range(5)) 
#         # ref = str(nets[0]+'.'+stas[0])
#         # try:
#         #     ref_env = data_env_dict[reference]
#         # except:
#         #     ref_env = data_env_dict[ref]


#     ################### PHASE PICKING using retrained eqT ####################
#     for i in range(len(sta_available)):
#         stream = evt_data.select(station=sta_available[i])
#         if len(stream)==3:
#             annotations = eqt.annotate(stream)
#             print(annotations)
#         elif len(stream)==1:
#             # copy stream to 2 components, zero the missing data.
#             tr3 = stream[0].copy()
#             tr2 = stream[0].copy();tr2.stats.channel=stream[0].stats.channel[0:2]+"N"
#             tr1 = stream[0].copy();tr1.stats.channel=stream[0].stats.channel[0:2]+"E"
#             tr1.data=np.zeros(len(stream[0].data))
#             tr2.data=np.zeros(len(stream[0].data))
#             crap=obspy.Stream(traces=[tr1,tr2,tr3])
#             print(crap)

#         pcount = {"P":0, "S":0}
#         for ii in eqt.classify(stream)[0]:
#             pcount[ii.phase] += 1
#         print('--------------'*3)
#         print(f"{pcount['P']} P picks\n{pcount['S']} S picks")

#         # print(__annotations__)
#         print(annotations[0])
#         annotations[0].stats.sampling_rate=40
#         t2=annotations[0].times()
#         ax.plot(t2,annotations[1].data+i*1.5, color = 'r')
#         ax.plot(t2,annotations[2].data+i*1.5, color = 'b')



#     plt.show()


In [ ]:
print(evt_data)
tr3 = evt_data[-1].copy()
tr2 = evt_data[-1].copy();tr2.stats.channel=evt_data[-1].stats.channel[0:2]+"N"
tr1 = evt_data[-1].copy();tr1.stats.channel=evt_data[-1].stats.channel[0:2]+"E"
tr1.data=np.zeros(len(evt_data[-1].data))
tr2.data=np.zeros(len(evt_data[-1].data))
crap=obspy.Stream(traces=[tr1,tr2,tr3])
# crap[0].stats.channel=crap[0].stats.channel[0:2]+"I"
# crap[1].stats.channel=crap[0].stats.channel[0:2]+"H"
# crap[2].stats.channel=crap[0].stats.channel[0:2]+"Z"

# crap[0].data=np.zeros(len(crap[1].data))


print(crap[0].data)

# print(crap[2].data)

In [ ]:
for n in range(6290,6291):    
    event_ID = str(evt_id[n])
    time = UTCDateTime(start_time[n])
    if net != 'CN' and evt_id[n]!=evt_id[n-1]:
        reference = str(net[n]+'.'+sta[n])
        try:
            associated_volcano = df[df['Station']== sta[n]]['Volcano_Name'].values[0]
        except: 
            pass
        print(associated_volcano)
        # if associated_volcano == 'Mt_Rainier':
            
        #get info for stations within 50km of volcano that event ocurred at
        stations = df[df['Volcano_Name'] == associated_volcano]['Station'].values.tolist()
        networks = df[df['Volcano_Name'] == associated_volcano]['Network'].values.tolist()
        latitudes = df[df['Volcano_Name'] == associated_volcano]['Latitude'].values.tolist()
        longitudes = df[df['Volcano_Name'] == associated_volcano]['Longitude'].values.tolist()
        elevations = df[df['Volcano_Name']== associated_volcano]['Elevation'].values.tolist()

        if stations.count("LON")>0 and stations.count("LO2")>0:
            index = stations.index("LO2")
            del stations[index]
            del networks[index]
            del latitudes[index]
            del longitudes[index]
            del elevations[index]

            #Download all waveforms for that event based on stations and times
            bulk = [] 
            for m in range(0, len(networks)):
                bulk.append([networks[m], stations[m], '*', '*', time-t_before, time+t_after])
            st = client2.get_waveforms_bulk(bulk)

            #remove unwanted data
            for tr in st:
                cha = tr.stats.channel
                if cha[0:2] != 'BH' and cha[0:2] != 'EH' and cha[0:2] != 'HH':
                    st.remove(tr)
                try:
                    if len(tr.data)/tr.stats.sampling_rate < 239.9:
                        st.remove(tr)
                except:
                    pass

            #resampling the data to 40Hz for each trace
            st = resample(st,fs) 

            #Plotting all traces for one event with channel z, SNR>10, and bandpasses between 2-12Hz
            SNR,SNR_weight, no_weight,stas,nets,max_amp_times,durations,data_env_dict,t_diff = [],[],[],[],[],[],[],{},{}
            fig = plt.figure(figsize = (11,8), dpi=200)
            fig.suptitle('evtID:UW'+ event_ID+associated_volcano)
            plt.rcParams.update({'font.size': 20})
            ax1 = plt.subplot(1,1,1)
            iplot = 0
            for i,ii in enumerate(st):
                network = ii.stats.network
                station = ii.stats.station
                ii.detrend(type = 'demean')
                ii.filter('bandpass',freqmin=2.0,freqmax=12.0,corners=2,zerophase=True)
                cha = ii.stats.channel
                starttime = ii.stats.starttime
                max_amp_time = np.argmax(ii.data)/fs
                signal_window = ii.copy()
                noise_window = ii.copy()
                signal_window.trim(starttime + t_before - 20, starttime + t_before - 20 + window)
                noise_window.trim(starttime + t_before - window -10, starttime + t_before - 10)
                snr = (20 * np.log(np.percentile(np.abs(signal_window.data),pr) 
                               / np.percentile(np.abs(noise_window.data),pr))/np.log(10))

                if cha[-1] == 'Z' and snr>thr and 100<max_amp_time<200:
                    t = ii.times()
                    t_diff[network+'.'+station] = starttime-time 
                    # enveloping the data 
                    data_envelope = obspy.signal.filter.envelope(ii.data[115*fs:150*fs])
                    data_envelope /= np.max(data_envelope)
                    data_envelope += iplot*1.5
                    # finding the time of max amplitude of each event
                    max_amp_times.append(max_amp_time)
                    max_amp = np.max(ii.data)      
                    # creating envelope data dictionary to calculate picktimes
                    data_envelope = obspy.signal.util.smooth(data_envelope, smooth_length)
                    data_env_dict[network+'.'+station]= data_envelope
                    b,e = 115,150
                    ax1.plot(t[b*fs:e*fs],ii.data[b*fs:e*fs]/np.max(np.abs(ii.data))+iplot*1.5)
                    ax1.plot(t[115*fs:150*fs], data_envelope, color = 'k')
                    ax1.set_xlabel('time (seconds)')
                    ax1.set_xlim([b,e])
                    ax1.set_yticks([])
                    plt.text(t[e*fs], iplot*1.5, 'SNR:'+str(int(snr)))
                    plt.text(t[b*fs], iplot*1.5, station)
                    iplot = iplot+1
                    stas.append(ii.stats.station)
                    nets.append(ii.stats.network)
                    SNR.append(snr)
                    SNR_weight.append(int(snr))
                    no_weight.append(1)
                else:
                    st.remove(ii)

            if len(st)<4:  
                continue

            # get peak frequency of each event
            # read and preprocess data
            st.taper(max_percentage=0.01,max_length=20)
            st.trim(starttime=time-20,endtime=time+30) 

            # make plot of spectra
            char_freq, sharp_weight= [],[]
    #         fig,ax = plt.subplots(1,1,figsize=(11,7), dpi = 200)
            for i in range(len(stas)):
                data = st.select(station=stas[i],component="Z")[0].data*100
                f,psd=scipy.signal.welch(data,fs=st[0].stats.sampling_rate,nperseg=81,noverlap=1)
                #just get the frequencies within the filter band
                above_low_cut = [f>low_cut]
                below_high_cut = [f<high_cut]
                in_band = np.logical_and(above_low_cut,below_high_cut)[0]
                f = f[in_band]
                psd = psd[in_band]

                # calculate characteristic frequency and report
                char_freq_max = f[np.argmax(psd)]
                char_freq_mean= np.sum(psd*f)/np.sum(psd)
                psd_cumsum = np.cumsum(psd)
                psd_sum = np.sum(psd)
                char_freq_median = f[np.argmin(np.abs(psd_cumsum-psd_sum/2))]
                char_freq.append(char_freq_mean)

                plt.rcParams.update({'font.size': 20})
    #             ax.plot(f,psd,label=stas[i],linewidth=2)
    #             ax.set_xscale('log')
    #             ax.set_yscale('log')
    #             ax.grid('True')
    #             ax.set_xlabel('Frequency [Hz]')
    #             ax.set_ylabel('PSD [$(mm/s)^2$/Hz]')
    #             ax.vlines(char_freq_mean,ymin=np.min(psd)/10,ymax=np.max(psd)*10,linestyle="--",colors=colors[i])

                # weighting the data by the spikiness of the PSD vs frequency graphs
                ratio = (np.mean(psd)/np.max(psd))
                sharp_weight.append(int(1/(ratio**2)*20))

    #         ax.legend() 
    #         plt.savefig('psd'+event_ID+associated_volcano+'.png')

            lats, lons, elevs, r, theta = ([] for i in range(5)) 
            ref = str(nets[0]+'.'+stas[0])
            try:
                ref_env = data_env_dict[reference]
            except:
                ref_env = data_env_dict[ref]

            ############ PHASE PICKING ############################
            # calculating the picktimes and shift in arrival times using envelope cross_correlation
            pick_times, offsets, starttimes = pick_time(time, ref_env, data_env_dict,st,t_diff, t_before, fs) #calculate picktimes
            shifts, vals = shift(pick_times, offsets, starttimes, t_diff)

            iplot = 0 
            durations = []
            for i in range(len(stas)):
                max_amp_time = max_amp_times[i]
                duration = (max_amp_time-vals[i])*2
                durations.append(duration)
                ax1.vlines(vals[i], ymin = iplot*1.5-.5, ymax = iplot*1.5+.5, color = colors[i])
                #plt.text(t[110*fs], iplot*1.5, 'duration:'+str(int(duration))+'s')
                a = stations.index(stas[i])
                lats.append(latitudes[a])
                lons.append(longitudes[a])
                elevs.append(elevations[a])
                iplot = iplot+1
            avg_duration = np.mean(durations)
            plt.savefig('wiggles'+event_ID+associated_volcano+'.png')



            ############ LOCATION ############################
            # input necessary data for grid search
            arrivals = shifts
            sta_lats = lats
            sta_lons= lons

            # define grid origin in lat,lon and grid dimensions in m
            lat_start = volc_grid[associated_volcano][0]
            lon_start = volc_grid[associated_volcano][1]
            side_length = volc_grid[associated_volcano][2]

            # create the grid of locations
            sta_x = []
            sta_y = []
            for i in range(len(sta_lats)):
                x_dist = distance.distance([lat_start,lon_start],[lat_start,sta_lons[i]]).m
                y_dist = distance.distance([lat_start,lon_start],[sta_lats[i],lon_start]).m
                sta_x.append(x_dist)
                sta_y.append(y_dist)
            x_vect = np.arange(0, side_length, step)
            y_vect = np.arange(0, side_length, step)
            t0 = np.arange(0,np.max(arrivals),t_step)

            # carry out the gridsearch weighted by SNR
            weight = np.array(SNR_weight)/np.max(SNR_weight)
            rss_mat = gridsearch(t0,x_vect,y_vect,sta_x,sta_y,1000,arrivals,weight)
            loc_idx_snr = np.unravel_index([np.argmin(rss_mat)], rss_mat.shape)

            # gridsearch with no weight
            weight = [1 for i in range(len(SNR_weight))]
            rss_mat = gridsearch(t0,x_vect,y_vect,sta_x,sta_y,1000,arrivals,weight)
            loc_idx = np.unravel_index([np.argmin(rss_mat)], rss_mat.shape)

            # gridsearch weighted with SNR and Slope
            # plot DEM
            # gives the lower left grid point in the grid search
            # gives the left right, bottom, top of the grid
            left, right = r_dem_data_dict[associated_volcano]['left'],r_dem_data_dict[associated_volcano]['right']
            bottom, top = r_dem_data_dict[associated_volcano]['bottom'],r_dem_data_dict[associated_volcano]['top']

            crs = dem_data_dict[associated_volcano]['crs']
            data = dem_data_dict[associated_volcano]['data']
            volc = rd.rdarray(data, no_data=-9999)
            slope = rd.TerrainAttribute(volc,attrib = 'slope_riserun')
            info = volc_lat_lon[associated_volcano]
            p2 = Proj(crs,preserve_units=False)
            p1 = Proj(proj='latlong',preserve_units=False)
            # gives the lower left grid point in the grid search
            left_x,bottom_y = transform(p1,p2,volc_grid[associated_volcano][1],volc_grid[associated_volcano][0]) # p1,p2,lon,lat
            # gives the left right, bottom, top of the grid
            grid_bounds = [left_x, left_x+volc_grid[associated_volcano][2], bottom_y, bottom_y+volc_grid[associated_volcano][2]]
            left, right = r_dem_data_dict[associated_volcano]['left'],dem_data_dict[associated_volcano]['right']
            bottom, top = r_dem_data_dict[associated_volcano]['bottom'],dem_data_dict[associated_volcano]['top']

            a = int((left_x-left)/10)
            b = a+2500
            c = (slope.shape[0] - int((bottom_y-bottom)/10))-2500
            d = slope.shape[0] - int((bottom_y-bottom)/10)

            x = np.arange(a,b,1)
            y = np.arange(c,d,1)

            x2 = np.arange(a,b,10) # every 100m
            y2 = np.arange(c,d,10) # every 100m

            slope_data = np.array(slope[c:d,a:b])


            slope_norm1 = slope_data/np.max(slope_data)

            slope_interp_mat = RectBivariateSpline(y,x,slope_norm1, s = 0)
            interp = slope_interp_mat(x2,y2)/np.max(slope_interp_mat(x2,y2))*0.9+.1

            rss_mat_slope = np.multiply(rss_mat[loc_idx[0],:,:],interp)
            loc_idx_slope = np.unravel_index([np.argmin(rss_mat_slope)], rss_mat_slope.shape)
            loc_lat_slope, loc_lon_slope, test_d = location(x_vect[loc_idx_slope[1]], y_vect[loc_idx_slope[2]], lat_start, lon_start)

            # plot heatmap
    #         fig,ax = plt.subplots(1,1,figsize=(8,8), dpi = 200)
    #         ax.scatter(x_vect[loc_idx[1]],y_vect[loc_idx[2]],s=100,marker='*',c='r')
    #         im = ax.imshow(np.log10(rss_mat[loc_idx[0],:,:].T),origin="lower",extent=[0,side_length,0,side_length])
    #         ax.set_ylabel('(m)')
    #         ax.set_ylabel('(m)')
    #         cbar = plt.colorbar(im)
    #         cbar.ax.tick_params()
    #         cbar.set_label('RMS error on location', rotation=270)
    #         plt.savefig('heatmap'+ event_ID+associated_volcano+'.png')


            # find the latitude and longitude of the location index
            loc_lat, loc_lon, d = location(x_vect[loc_idx[1]], y_vect[loc_idx[2]], lat_start, lon_start)
            err_thr = np.min(np.log10(rss_mat))+.05
            thr_array = np.argwhere(np.log10(rss_mat)<err_thr)
            diameter = error_diameter(thr_array)

            break
            # calculating azimuth for each station with respect to the middle of the volcano
            for i in range(len(stas)):
                u,b,c = (gps2dist_azimuth(loc_lat, loc_lon, lats[i], lons[i], a=6378137.0, f=0.0033528106647474805))
                r.append(u)
                theta.append(b)

            bin1,bin2,bin3 = [],[],[]
            for i in theta:
                if 0<=i<=120:
                    bin1.append(i)
                if 121<=i<=240:
                    bin2.append(i)
                if 241<=i<=360:
                    bin3.append(i)

            if bin1 == [] or bin2 == [] or bin3 == []:
                continue

            #manipulating the data
            data = {'azimuth_deg':theta, 'freq':char_freq, 'station':stas, 'distance_m':r, 
                    'weight':sharp_weight, 'SNR':SNR, 'colors':colors[0:len(stas)]}
            DF = pd.DataFrame(data, index = None)
            DF2 = DF.sort_values('azimuth_deg')

            #Taking out stations that are too close to the location when looking at azimuth 
            drops = []
            for i in range(len(DF2)):
                value = DF2.loc[i,'distance_m']
                if value < az_thr:
                    drops.append(i)
            DF3 = DF2.drop(drops)
            y_data =  DF3["freq"].values.tolist()
            Sta2 = DF3["station"].values.tolist()
            dist2 = DF3["distance_m"].values.tolist()
            spike_weight = DF3["weight"].values.tolist()
            SNR2 = DF3['SNR'].values.tolist()
            colors2 = DF3['colors'].values.tolist()
            x_data =  np.asarray(DF3["azimuth_deg"].values.tolist())
            x_points = np.linspace(0,360, 100)


            ################ DIRECTIVITY FIT ##################################
            #optimizing parameters to fit data to test_function
            params, params_covariance = optimize.curve_fit(test_func, np.deg2rad(x_data), y_data, p0=None)
            perr = np.sqrt(np.diag(params_covariance))
            std_deviation = str(round(perr[0],9))+','+str(round(perr[1],9))+','+str(round(perr[2],9))
            d = test_func(np.deg2rad(x_points), params[0], params[1], params[2])
            len_r = int(max(r))

            if params[0]<0:
                direction = params[1]+pi 
            else:
                direction = params[1]

            fmax = max(d)
            fmin = min(d)
            v = v_s*((fmax-fmin)/(fmax+fmin))



            # weight the data
            # title = 'Sharpness'
            v_sharp,direction_sharp,d_sharp = weight_data(x_data,y_data,sharp_weight,test_func,v_s,stas)

            # title = 'SNR'
            v_snr,direction_snr,d_snr = weight_data(x_data,y_data,SNR_weight,test_func,v_s,stas)  




            ############### PLOTS ###################
            #convert the direction from polar to cartesian coordinates
            dy = len_r*np.sin(direction)
            dx = len_r*np.cos(direction)     

            dy_sharp = len_r*np.sin(direction_sharp)
            dx_sharp = len_r*np.cos(direction_sharp)    


            dy_snr = len_r*np.sin(direction_snr)
            dx_snr = len_r*np.cos(direction_snr) 




            fig,ax = plt.subplots(1,1,figsize=(11,8), dpi = 200)
            fig.suptitle('Fitted Cosine Curves')       
            ax.set_ylabel('characteristic frequency(Hz)')
            ax.set_xlabel(('azimuth(degrees)'))
            for i in range (0,len(Sta2)):
                ax.scatter(x_data[i], y_data[i], s = (SNR_weight[i]**2),label=Sta2[i], color = colors2[i])
            ax.plot(x_data,y_data, '--', label='rawdata')
            ax.plot(x_points, d, label = 'original')
            ax.plot(x_points, d_sharp, label = 'sharpness')
            ax.plot(x_points, d_snr, label = 'snr')
            ax.legend(loc='upper right', fontsize = 10)
            plt.grid(True)
            plt.savefig('curves_freq_data'+ event_ID+associated_volcano+'.png')

            #making plots of directivity and location
            crs = dem_data_dict[associated_volcano]['crs']
            data = dem_data_dict[associated_volcano]['data']
            info = volc_lat_lon[associated_volcano]
            p2 = Proj(crs,preserve_units=False)
            p1 = Proj(proj='latlong',preserve_units=False)
            # gives the lower left grid point in the grid search
            left_x,bottom_y = transform(p1,p2,volc_grid[associated_volcano][1],volc_grid[associated_volcano][0]) # p1,p2,lon,lat
            # gives the left right, bottom, top of the grid
            grid_bounds = [left_x, left_x+volc_grid[associated_volcano][2], bottom_y, bottom_y+volc_grid[associated_volcano][2]]
            left, right = dem_data_dict[associated_volcano]['left'],dem_data_dict[associated_volcano]['right']
            bottom, top = dem_data_dict[associated_volcano]['bottom'],dem_data_dict[associated_volcano]['top']

            # convert loc data onto the DEM data
            contour_x,contour_y = np.meshgrid(left_x+x_vect,bottom_y+y_vect)
            center_x, center_y = transform(p1,p2,info[1],info[0])
            loc_x,loc_y=transform(p1,p2,loc_lon,loc_lat)
            duration=avg_duration
            length_factor = duration/100

            fig,ax = plt.subplots(1,1,figsize=(8,11), dpi = 200)
            a = ax.imshow(data,extent=[left, right, bottom, top],cmap='gist_earth')
            contours = ax.contour(contour_x,contour_y,np.log10(rss_mat[int(loc_idx[0]),:,:].T),cmap='plasma')
            ax.scatter(center_x, center_y, s=100,marker='*',c='r')
            plt.arrow(loc_x,loc_y,dy*length_factor,dx*length_factor, color='w', width=170, label='no weight')
            plt.arrow(loc_x,loc_y,dy_sharp*length_factor,dx_sharp*length_factor, color='k', width=170, label='sharpness')
            plt.arrow(loc_x,loc_y,dy_snr*length_factor,dx_snr*length_factor, color='m', width=170, label='snr')
            #plotting the stations on top of this as triangles
            for i, ii in enumerate(stas):
                sta_x,sta_y = transform(p1,p2,lons[i],lats[i])
                if left+info[3]<sta_x<right-info[4] and bottom+info[5]<sta_y<top-info[6]:
                    ax.plot(sta_x,sta_y, c='k', marker="^")
                    ax.text(sta_x,sta_y,ii, c='k', fontsize = 15)

            #getting lat and lon tick marks on the axis
            tick_lons = lat_lon_dict[associated_volcano]['tick_lons']
            tick_lats = lat_lon_dict[associated_volcano]['tick_lats']
            ticks_x = []
            ticks_y = []
            for i in range(len(tick_lons)):
                tick_x,tick_y=transform(p1,p2,tick_lons[i],tick_lats[i])
                ticks_x.append(tick_x)
                ticks_y.append(tick_y)
                tick_lons[i]=str(tick_lons[i])
                tick_lats[i]=str(tick_lats[i])
            divider = make_axes_locatable(ax)
            cax1 = divider.append_axes('right', size='4%', pad=0.1)
            cax2 = divider.append_axes('right', size='4%', pad=1.3)
            ax.set_title('Location and Directivity', fontsize = 20)
            ax.set_xlabel('longitudes(DD)', fontsize = 15)
            ax.set_ylabel('latitudes(DD)', fontsize = 15)
            ax.set_xticks(ticks_x)
            ax.set_xticklabels(tick_lons, fontsize = 15)
            ax.set_yticks(ticks_y)
            ax.set_yticklabels(tick_lats, fontsize = 15)
            ax.clabel(contours)
            cbar = plt.colorbar(a, cax=cax1)
            cbar.ax.tick_params(labelsize=10)
            cbar.set_label('elevation(m)\n', rotation=270, labelpad = 13, fontsize = 15)
            cbar2 = plt.colorbar(contours, cax=cax2)
            cbar2.ax.tick_params(labelsize=10)
            cbar2.set_label('RMS error on location\n', rotation=270, labelpad = 13,fontsize = 15)
            ax.set_xlim(left+info[3],right-info[4])
            ax.set_ylim(bottom+info[5],top-info[6])
            ax.legend(fontsize = 12)
            plt.tight_layout()
            plt.savefig('loc_direction'+ event_ID+associated_volcano+'.png',bbox_inches="tight")

            # make a dataframe of the data
            evt_data = evt_data.append({'event_ID':event_ID, 
                        'location_latitude': loc_lat,
                        'location_longitude': loc_lon,
                        'location_uncertainty(m)':diameter/10,
                        'origin_time': min(offsets)-int(loc_idx[0]),
                        'direction(degrees)':np.rad2deg(direction),
                        'direction_sharpness(degrees)':np.rad2deg(direction_sharp),
                        'direction_snr(degrees)':np.rad2deg(direction_snr),
                        'duration(sec)':avg_duration,
                        'params_std_deviation':std_deviation, 
                        'velocity(m/s)':v, 
                        'number_of_stations':len(stas)}, ignore_index = True)

            dict_temp = {}
            for i in range(len(stas)):
                dict_temp[stas[i]] = char_freq[i]
            print(dict_temp)    
            sta_freq = sta_freq.append(dict_temp,ignore_index = True)

            evt_data.to_csv('~/surface_events/Event_Data.csv', index=False)
            sta_freq.to_csv('~/surface_events/Station_frequency_data.csv', index=False)
    #     except:
    #         reject_evts = reject_evts.append({'event_ID':[event_ID]}, ignore_index = True)
    #         reject_evts.to_csv('~/surface_events/Rejects5.csv', index=False)
    #         continue


In [ ]:
#DEM data 
r_dem_data_dict = {}
name = 'Mt_Rainier'
if volc_lat_lon[name][0]>46:
    dem = rio.open('Data/DEM_data/'+str(name)+'/'+str(name)+'1.tif') #washington volcanoes
    dem_array = dem.read(1).astype('float64')
    dem_array[dem_array == -32767] = np.nan #gets rid of edge effects
    crs = dem.crs

r_dem_data_dict[name]={'data':dem_array, 'crs':crs, 'left':dem.bounds[0], 'right':dem.bounds[2], 'bottom':dem.bounds[1], 'top':dem.bounds[3]}

In [ ]:
# Determine the avg slope value within one step of the grid search model
associated_volcano = 'Mt_Rainier'
step_gs = 100 # grid search model step size is 100m

# plot DEM
crs = r_dem_data_dict[associated_volcano]['crs']
data = r_dem_data_dict[associated_volcano]['data']
volc = rd.rdarray(data, no_data=-9999)
slope = rd.TerrainAttribute(volc,attrib = 'slope_riserun')
info = volc_lat_lon[associated_volcano]
p2 = Proj(crs,preserve_units=False)
p1 = Proj(proj='latlong',preserve_units=False)
# gives the lower left grid point in the grid search
left_x,bottom_y = transform(p1,p2,volc_grid[associated_volcano][1],volc_grid[associated_volcano][0]) # p1,p2,lon,lat
# gives the left right, bottom, top of the grid
grid_bounds = [left_x, left_x+volc_grid[associated_volcano][2], bottom_y, bottom_y+volc_grid[associated_volcano][2]]
left, right = r_dem_data_dict[associated_volcano]['left'],r_dem_data_dict[associated_volcano]['right']
bottom, top = r_dem_data_dict[associated_volcano]['bottom'],r_dem_data_dict[associated_volcano]['top']
contour_x,contour_y = np.meshgrid(left_x+x_vect,bottom_y+y_vect)
center_x, center_y = transform(p1,p2,info[1],info[0])
loc_x,loc_y=transform(p1,p2,loc_lon,loc_lat)

# looking at where the grid is:
x = np.linspace(left_x, left_x+25000, 10)
y = np.linspace(bottom_y, bottom_y+25000, 10)

In [ ]:
a = int((left_x-left)/10)
b = a+2500
c = (slope.shape[0] - int((bottom_y-bottom)/10))-2500
d = slope.shape[0] - int((bottom_y-bottom)/10)

x = np.arange(a,b,1)
y = np.arange(c,d,1)

x2 = np.arange(a,b,10) # every 100m
y2 = np.arange(c,d,10) # every 100m

slope_data = np.array(slope[c:d,a:b])

In [ ]:
slope_norm1 = slope_data/np.max(slope_data)

slope_interp_mat = RectBivariateSpline(y,x,slope_norm1, s = 0)
interp = slope_interp_mat(x2,y2)/np.max(slope_interp_mat(x2,y2))*0.9+.1

rss_mat_slope = np.multiply(rss_mat[loc_idx[0],:,:],interp)
loc_idx_slope = np.unravel_index([np.argmin(rss_mat_slope)], rss_mat_slope.shape)
loc_lat_slope, loc_lon_slope, j = location(x_vect[loc_idx_slope[1]], y_vect[loc_idx_slope[2]], lat_start, lon_start)


In [ ]:
plt.rcParams.update({'font.size': 5})

# weighted by SNR
weight = np.array(SNR_weight)/np.max(SNR_weight)
rss_mat = gridsearch(t0,x_vect,y_vect,sta_x,sta_y,1000,arrivals,weight)
loc_idx = np.unravel_index([np.argmin(rss_mat)], rss_mat.shape)
loc_lat, loc_lon, j = location(x_vect[loc_idx[1]], y_vect[loc_idx[2]], lat_start, lon_start)

x2 = np.linspace(a,b,250)
y2 = np.linspace(c,d,250)

# weighted by SNR and Slope
rss_mat_slope = np.multiply(rss_mat[loc_idx[0],:,:],interp)
loc_idx_slope = np.unravel_index([np.argmin(rss_mat_slope)], rss_mat_slope.shape)
loc_lat_slope, loc_lon_slope, j = location(x_vect[loc_idx_slope[1]], y_vect[loc_idx_slope[2]], lat_start, lon_start)

# unweighted
no_weight = [1,1,1,1,1,1,1,1]
rss_mat_nw = gridsearch(t0,x_vect,y_vect,sta_x,sta_y,1000,arrivals,no_weight)
loc_idx_nw = np.unravel_index([np.argmin(rss_mat_nw)], rss_mat_nw.shape)
loc_lat_nw, loc_lon_nw, j = location(x_vect[loc_idx_nw[1]], y_vect[loc_idx_nw[2]], lat_start, lon_start)


# plotting the results
fig,ax = plt.subplots(3,1,figsize=(6,9), dpi = 180)

ax[0].set_title('weighted by SNR: '+str(round(loc_lat,3))+','+str(round(loc_lon,3)))
ax[0].scatter(x_vect[loc_idx[1]],y_vect[loc_idx[2]],s=100,marker='*',c='r')
im0 = ax[0].imshow(np.log10(rss_mat[loc_idx[0],:,:].T),origin="lower",extent=[0,side_length,0,side_length])
#im0 = ax[0].imshow(rss_mat[loc_idx[0],:,:].T,origin="lower",extent=[0,side_length,0,side_length])
ax[0].set_ylabel('(m)')
ax[0].set_ylabel('(m)')
cbar0 = plt.colorbar(im0, ax = ax[0])
cbar0.ax.tick_params()
cbar0.set_label('RMS error on location', rotation=270)

ax[1].set_title('weighted by SNR and Slope: '+str(round(loc_lat_slope,3))+','+str(round(loc_lon_slope,3)))
ax[1].scatter(x_vect[loc_idx_slope[1]],y_vect[loc_idx_slope[2]],s=100,marker='*',c='r')
im1 = ax[1].imshow(np.log10(rss_mat_slope[0,:,:].T),origin="lower",extent=[0,side_length,0,side_length])
#im1 = ax[1].imshow(rss_mat_slope[0,:,:].T,origin="lower",extent=[0,side_length,0,side_length])
ax[1].set_ylabel('(m)')
ax[1].set_ylabel('(m)')
cbar1 = plt.colorbar(im1, ax = ax[1])
cbar1.ax.tick_params()
cbar1.set_label('RMS error on location', rotation=270)

ax[2].set_title('unweighted: '+str(round(loc_lat_nw,3))+','+ str(round(loc_lon_nw,3)))
ax[2].scatter(x_vect[loc_idx_nw[1]],y_vect[loc_idx_nw[2]],s=100,marker='*',c='r')
im2 = ax[2].imshow(np.log10(rss_mat_nw[0,:,:].T),origin="lower",extent=[0,side_length,0,side_length])
#im2 = ax[2].imshow(rss_mat_nw[0,:,:].T,origin="lower",extent=[0,side_length,0,side_length])
ax[2].set_ylabel('(m)')
ax[2].set_ylabel('(m)')
cbar2 = plt.colorbar(im2, ax = ax[2])
cbar2.ax.tick_params()
cbar2.set_label('RMS error on location', rotation=270)